# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [70]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [71]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cavalaire-sur-mer,43.1740,6.5272,2.40,45,0,2.30,FR,1735244747
1,1,grytviken,-54.2811,-36.5092,5.90,90,100,0.72,GS,1735244748
2,2,mata-utu,-13.2816,-176.1745,28.41,71,99,6.09,WF,1735244750
3,3,bethel,41.3712,-73.4140,2.79,52,0,1.54,US,1735244703
4,4,hawaiian paradise park,19.5933,-154.9731,25.50,75,75,4.12,US,1735244752


In [72]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     555 non-null    int64  
 1   City        555 non-null    object 
 2   Lat         555 non-null    float64
 3   Lng         555 non-null    float64
 4   Max Temp    555 non-null    float64
 5   Humidity    555 non-null    int64  
 6   Cloudiness  555 non-null    int64  
 7   Wind Speed  555 non-null    float64
 8   Country     553 non-null    object 
 9   Date        555 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 43.5+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [96]:


# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE
mapplot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "Humidity",
    cmap = "Plasma",
    hover_cols = ["Humidity", "Max Temp"]

)

mapplot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [74]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[city_data_df["Humidity"] < 75]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cavalaire-sur-mer,43.1740,6.5272,2.40,45,0,2.30,FR,1735244747
2,2,mata-utu,-13.2816,-176.1745,28.41,71,99,6.09,WF,1735244750
3,3,bethel,41.3712,-73.4140,2.79,52,0,1.54,US,1735244703
5,5,lebu,-37.6167,-73.6500,16.90,74,14,5.63,CL,1735244597
9,9,puerto natales,-51.7236,-72.4875,15.25,38,40,11.83,CL,1735244758


### Step 3: Create a new DataFrame called `hotel_df`.

In [75]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = None
# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,cavalaire-sur-mer,43.1740,6.5272,2.40,45,0,2.30,FR,1735244747,None
2,2,mata-utu,-13.2816,-176.1745,28.41,71,99,6.09,WF,1735244750,None
3,3,bethel,41.3712,-73.4140,2.79,52,0,1.54,US,1735244703,None
5,5,lebu,-37.6167,-73.6500,16.90,74,14,5.63,CL,1735244597,None
9,9,puerto natales,-51.7236,-72.4875,15.25,38,40,11.83,CL,1735244758,None
...,...,...,...,...,...,...,...,...,...,...,...
539,539,al burayqah,30.4062,19.5739,15.60,69,0,8.72,LY,1735245396,None
541,541,bakanas,44.8081,76.2772,-2.73,72,100,4.40,KZ,1735245399,None
544,544,maryville,35.7565,-83.9705,15.08,59,75,0.89,US,1735245124,None
549,549,bako,5.7833,36.5667,22.21,48,81,1.02,ET,1735245408,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [76]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{long},{radius}"
    params["bias"] = f"proximity:{lat},{long}"

    print(params)

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'apiKey': '3de62f78981048e8a4c831d7565da416', 'categories': 'accommodation.hotel', 'filter': 'circle:43.174,6.5272,10000', 'bias': 'proximity:43.174,6.5272'}
{'type': 'FeatureCollection', 'features': []}
cavalaire-sur-mer - nearest hotel: No hotel found
{'apiKey': '3de62f78981048e8a4c831d7565da416', 'categories': 'accommodation.hotel', 'filter': 'circle:-13.2816,-176.1745,10000', 'bias': 'proximity:-13.2816,-176.1745'}
{'type': 'FeatureCollection', 'features': []}
mata-utu - nearest hotel: No hotel found
{'apiKey': '3de62f78981048e8a4c831d7565da416', 'categories': 'accommodation.hotel', 'filter': 'circle:41.3712,-73.414,10000', 'bias': 'proximity:41.3712,-73.414'}
{'type': 'FeatureCollection', 'features': []}
bethel - nearest hotel: No hotel found
{'apiKey': '3de62f78981048e8a4c831d7565da416', 'categories': 'accommodation.hotel', 'filter': 'circle:-37.6167,-73.65,10000', 'bias': 'proximity:-37.6167,-73.65'}
{'type': 'FeatureCollection', 'features': []}
lebu - nea

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,cavalaire-sur-mer,43.1740,6.5272,2.40,45,0,2.30,FR,1735244747,No hotel found
2,2,mata-utu,-13.2816,-176.1745,28.41,71,99,6.09,WF,1735244750,No hotel found
3,3,bethel,41.3712,-73.4140,2.79,52,0,1.54,US,1735244703,No hotel found
5,5,lebu,-37.6167,-73.6500,16.90,74,14,5.63,CL,1735244597,No hotel found
9,9,puerto natales,-51.7236,-72.4875,15.25,38,40,11.83,CL,1735244758,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
539,539,al burayqah,30.4062,19.5739,15.60,69,0,8.72,LY,1735245396,Aldufufa Tourist Resort
541,541,bakanas,44.8081,76.2772,-2.73,72,100,4.40,KZ,1735245399,No hotel found
544,544,maryville,35.7565,-83.9705,15.08,59,75,0.89,US,1735245124,No hotel found
549,549,bako,5.7833,36.5667,22.21,48,81,1.02,ET,1735245408,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [90]:


# Configure the map plot

hotel_found = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]


mapplot2 = hotel_found.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    tiles = "OSM",
    size = 3,
    scale = 1,
    color = "red",
    hover_cols = ["Hotel Name"]

)

mapplot2

# Display the map
#

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)